In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont
from scipy.interpolate import interp1d

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')
RESULT_O_DIR = os.path.join(ASPECT_LAB_DIR, 'results', "ConradHager1999")
TwoDSubduction_DIR = os.environ['TwoDSubduction_DIR']

sys.path.append(os.path.join(ASPECT_LAB_DIR))

from shilofue.Analytics import HAGER_CONRAD1999
from shilofue.TwoDSubduction0.Group import CASE_SUMMARY, PlotGroupDiagram
from shilofue.TwoDSubduction0.PlotVisit import VISIT_OPTIONS

In [ ]:
yr = 365 * 24 * 3600

strong_strength = 500e6
intermediate_strength = 300e6
weak_strength = 100e6

ref_strain_rate = 1e-14
strong_visc = strong_strength / 2.0 / ref_strain_rate
intermediate_visc = intermediate_strength / 2.0 / ref_strain_rate
weak_visc = weak_strength / 2.0 / ref_strain_rate

print("strong_visc: ", strong_visc)
print("intermediate_visc: ", intermediate_visc)
print("weak_visc: ", weak_visc)

HC1999_strong = HAGER_CONRAD1999(1.5e23) # 1e23 - their original value; 1.5e23 - match the case; strong_visc - value from ref_strain_rate
HC1999_intermediate = HAGER_CONRAD1999(2e22)  # intermediate_visc - value from ref_strain_rate
HC1999_weak = HAGER_CONRAD1999(3e21) # 3e21 - their original value; weak_visc - value from ref_strain_rate

'''
eta_sz = 2.5e20 # Pa * s
Vc_strong = HC1999_strong.ComputeConvergence(eta_sz)
Vc_weak = HC1999_weak.ComputeConvergence(eta_sz)
Vc_strong * yr * 100.0
Vc_weak * yr * 100.0
'''


In [ ]:
eta_m = 2.5e20

'''
L_l - the slab length
h_l - the slab thickness
R_l - the slab bending curvature
'''
L_l = 660e3 # m
h_l = 100e3 # m
R_l = 350e3 # m

zeta_f = 150e3 / 7.5e3 # aspect ratio of the shear zone

i = 0
N = 100
etas_sz = np.ones(N)
Vcs0 = np.zeros(N)
Vcsi = np.zeros(N)
Vcs1 = np.zeros(N)
log_etas = np.linspace(-3.0, 2.0, N)

for i in range(N):
    eta_sz = eta_m * 10**log_etas[i]
    etas_sz[i] = eta_sz
    Vc_strong = HC1999_strong.ComputeConvergence(L_l, h_l, R_l, eta_sz, zeta_f)
    Vc_intermediate = HC1999_intermediate.ComputeConvergence(L_l, h_l, R_l, eta_sz, zeta_f)
    Vc_weak = HC1999_weak.ComputeConvergence(L_l, h_l, R_l, eta_sz, zeta_f)
    Vcs0[i] = Vc_strong
    Vcsi[i] = Vc_intermediate
    Vcs1[i] = Vc_weak



In [ ]:
group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_2') # different sz properties

o_path = os.path.join(group_dir, 'case_summary_sz_visc.txt')

Case_Summary = CASE_SUMMARY(VISIT_OPTIONS=VISIT_OPTIONS)

# import old result if it exists 
if os.path.isfile(o_path):
    Case_Summary.import_txt(o_path)

V_plate_avgs = Case_Summary.export("V_plate_avgs")
V_ov_plate_avgs = Case_Summary.export("V_ov_plate_avgs")
sz_viscs = Case_Summary.export("sz_viscs")
slab_strs = Case_Summary.export("slab_strs")

V_convs = V_plate_avgs - V_ov_plate_avgs

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(etas_sz / eta_m, Vcs0 * 100.0 * yr, 'r', label="strong")
ax.semilogx(etas_sz / eta_m, Vcsi * 100.0 * yr, 'b', label="intermediate")
ax.semilogx(etas_sz / eta_m, Vcs1 * 100.0 * yr, 'c', label="weak")
ax.grid()
ax.set_xlabel("V_sz/V_m")
ax.set_ylabel("Vc (cm / yr)")

mask0 = (abs((slab_strs - strong_strength) / strong_strength) < 1e-6)
mask1 = (abs((slab_strs - intermediate_strength) / intermediate_strength) < 1e-6)
mask2 = (abs((slab_strs - weak_strength) / weak_strength) < 1e-6)
ax.semilogx(sz_viscs[mask0] / eta_m, V_convs[mask0] * 100.0, 'ro')
ax.semilogx(sz_viscs[mask1] / eta_m, V_convs[mask1] * 100.0, 'bo')
ax.semilogx(sz_viscs[mask2] / eta_m, V_convs[mask2] * 100.0, 'co')
ax.legend()
fig_path = os.path.join(RESULT_O_DIR, "Behr_etal_2022_fig9_Ll%.1f_hl%.1f_Rl%.1f_zetaf%.1f_compare.pdf" % (L_l/1e3, h_l/1e3, R_l/1e3, zeta_f))
fig.savefig(fig_path)
print("saved to %s" % fig_path)